In [2]:
# @title
from IPython.display import HTML

html_code = """
<div style="text-align: center;
            background-color: rgba(246, 211, 36, 0.8);">
    <h1 style="color: black;
               font-size: 180%;
               font-family: Nexa;
               letter-spacing: 0px;
               padding: 10px;">
        Entregable Semana 7
    </h1>
</div>

<h2 style="text-align: center;
           color: rgba(195, 172, 71, 1);
           font-family: Nexa;
           font-size: 180%;
           letter-spacing: 0px;">
Predicción de texto utilizando redes neuronales
</h2>

<div style="text-align: center;
            background-color: white;">
    <h1 style="color: black;
               font-size: 140%;
               font-family: Nexa;
               letter-spacing: 0.0px;
               padding: 0px;">
        <br><span style="font-weight:normal;">Melissa Fonseca Veitia</span><br>
         <br><span style="font-weight:normal;">Darwin Yadir Londoño Ochoa</span><br>
         <br><span style="font-weight:normal;">Samir Romero Cárdenas</span><br>
         <br><span style="font-weight:normal;">Ruber Alberto Barrios Rodríguez</span><br>

        <br>
        <span style="font-weight:bold;">Uniminuto</span><br>
        <span style="font-weight:bold;">Procesamiento de Lenguaje Natural</span><br>
        <br>
        <span style="font-weight:normal;">Docente: Claudia Marcela Ospina Mosquera</span><br>
        <br>
        Febrero - 2024<br>
        <br>
    </h1>
</div>


<hr style="border: 1px solid rgba(195, 172, 71, 1);"> <!-- Línea larga -->
"""

display(HTML(html_code))

#Actividad

 El estudiante, de manera colaborativa, elaborará un proyecto integrador sobre el uso de las redes neuronales recurrentes, en el cual tomará como base un código de predicción de textos, al finalizar, este debe predecir caracteres de un texto utilizando redes neuronales recurrentes y LSTM, con el objetivo de determinar las áreas de aplicación con base en las técnicas modernas de deep learning. Esta actividad le permitirá desarrollar resultados de aprendizaje de catalogar las redes recurrentes modernas y deducir las tendencias actuales de la IA.

In [1]:
import numpy as np
import keras
import matplotlib.pyplot as plt
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
import sys
import random
import io
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Cargamos un texto diferente al quijote almacenado en Drive
with open("/content/drive/MyDrive/mitosyleyendas.txt", "r") as f:
    text = f.read().lower() #aplicamos minúsculas a todo el documento
print(text)

dice la leyenda que en las calles de ocaña las gentes podían ver el fantasma del jinete negro, don antón garcía, caballero de la época colonial.

el espectro iba vestido de negro, portaba un sombrero y unas alas anchas. de sus hombros caía una capa oscura y larga que le cubría todo el cuerpo. su aspecto causaba espanto y pavor a aquellos que se lo cruzaban.

cuentan que, en vida, hizo construir un lago en una de sus fincas, cerca del río magdalena, para lo cual empleó mucha servidumbre. le encantaban los caballos, y todas las noches se podía oír su caballo negro saltar por las oscuras calles del lugar.

cuando su esposa enfermó, don antón garcía le hizo una promesa a santa rita, patrona de los imposibles. sin embargo, esta promesa se le olvidó y, cuando don antón garcía falleció, san pedro le ordenó acudir cada noche a visitar el santuario de santa rita, hasta la consumación de los siglos.
 


In [4]:
print("Longitud del texto: {}".format(len(text)))
print(text[0:500])

Longitud del texto: 904
dice la leyenda que en las calles de ocaña las gentes podían ver el fantasma del jinete negro, don antón garcía, caballero de la época colonial.

el espectro iba vestido de negro, portaba un sombrero y unas alas anchas. de sus hombros caía una capa oscura y larga que le cubría todo el cuerpo. su aspecto causaba espanto y pavor a aquellos que se lo cruzaban.

cuentan que, en vida, hizo construir un lago en una de sus fincas, cerca del río magdalena, para lo cual empleó mucha servidumbre. le encan


**Procesamiento de Datos**

In [5]:
chars=sorted(list(set(text)))
char_indices = dict((c,i) for i, c in enumerate(chars))
indice_char = dict((i, c) for i, c in enumerate(chars))

In [7]:
chars[:]

['\n',
 ' ',
 ',',
 '.',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'y',
 'z',
 'é',
 'í',
 'ñ',
 'ó']

In [6]:
#indice_char
print("Tiene",len(chars),"caracteres") # se obtienen 31 caracteres
char_indices

Tiene 31 caracteres


{'\n': 0,
 ' ': 1,
 ',': 2,
 '.': 3,
 'a': 4,
 'b': 5,
 'c': 6,
 'd': 7,
 'e': 8,
 'f': 9,
 'g': 10,
 'h': 11,
 'i': 12,
 'j': 13,
 'l': 14,
 'm': 15,
 'n': 16,
 'o': 17,
 'p': 18,
 'q': 19,
 'r': 20,
 's': 21,
 't': 22,
 'u': 23,
 'v': 24,
 'y': 25,
 'z': 26,
 'é': 27,
 'í': 28,
 'ñ': 29,
 'ó': 30}

In [28]:
# Definia el tamaño de las secuencias. Puede dejar este valor por defecto.
SEQ_LENGTH = 35
step=3
rawX = []
rawy = []

for i in range(0, len(text) - SEQ_LENGTH, step):#Ponemos text
    rawX.append(text[i: i+SEQ_LENGTH])
    rawy.append(text[i+SEQ_LENGTH])

In [29]:
print(rawX) #imprimimos lo que guardó cada vector
print(rawy)

['dice la leyenda que en las calles d', 'e la leyenda que en las calles de o', 'a leyenda que en las calles de ocañ', 'eyenda que en las calles de ocaña l', 'nda que en las calles de ocaña las ', ' que en las calles de ocaña las gen', 'e en las calles de ocaña las gentes', 'n las calles de ocaña las gentes po', 'as calles de ocaña las gentes podía', 'calles de ocaña las gentes podían v', 'les de ocaña las gentes podían ver ', ' de ocaña las gentes podían ver el ', ' ocaña las gentes podían ver el fan', 'aña las gentes podían ver el fantas', ' las gentes podían ver el fantasma ', 's gentes podían ver el fantasma del', 'entes podían ver el fantasma del ji', 'es podían ver el fantasma del jinet', 'podían ver el fantasma del jinete n', 'ían ver el fantasma del jinete negr', ' ver el fantasma del jinete negro, ', 'r el fantasma del jinete negro, don', 'l fantasma del jinete negro, don an', 'antasma del jinete negro, don antón', 'asma del jinete negro, don antón ga', 'a del jinete negro, don

In [30]:
n_sentences=len(rawX)
n_sentences

290

In [31]:
MAX_SEQUENCES = 500

perm = np.random.permutation(len(rawX)) #Permutar aleatoriamente una secuencia, o devolver un rango permutado.
rawX, rawy = np.array(rawX), np.array(rawy)
rawX, rawy = rawX[perm], rawy[perm]
rawX, rawy = list(rawX[:MAX_SEQUENCES]), list(rawy[:MAX_SEQUENCES])

print(len(rawX))

290


In [33]:
X = np.zeros((len(rawX), SEQ_LENGTH , len(chars)))
y = np.zeros((len(rawX), len(chars)))

In [34]:
for i, sentence in enumerate(rawX):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[rawy[i]]] = 1

In [35]:
X[:1]

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.]]])

In [36]:
y[:]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [37]:
# Definir el modelo
model = Sequential()
# Añadir una capa LSTM
model.add(LSTM(128, input_shape=(SEQ_LENGTH, len(chars))))
# Añadir una capa de dropout para evitar sobreajuste
model.add(Dropout(0.5))
# Añadir una capa densa con activación softmax para la salida
model.add(Dense(len(chars), activation='softmax'))
# Imprimir un resumen del modelo
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               81920     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 31)                3999      
                                                                 
Total params: 85919 (335.62 KB)
Trainable params: 85919 (335.62 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [38]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [41]:
model.fit(X,y,batch_size=128,epochs=250,verbose=1)

Epoch 1/250
3/3 [==============================] - 0s 11ms/step - loss: 2.7469 - accuracy: 0.2000
Epoch 2/250
3/3 [==============================] - 0s 16ms/step - loss: 2.6720 - accuracy: 0.2379
Epoch 3/250
3/3 [==============================] - 0s 12ms/step - loss: 2.6364 - accuracy: 0.2241
Epoch 4/250
3/3 [==============================] - 0s 13ms/step - loss: 2.6967 - accuracy: 0.2207
Epoch 5/250
3/3 [==============================] - 0s 11ms/step - loss: 2.6811 - accuracy: 0.2586
Epoch 6/250
3/3 [==============================] - 0s 12ms/step - loss: 2.6704 - accuracy: 0.2276
Epoch 7/250
3/3 [==============================] - 0s 8ms/step - loss: 2.6735 - accuracy: 0.2241
Epoch 8/250
3/3 [==============================] - 0s 8ms/step - loss: 2.7256 - accuracy: 0.2172
Epoch 9/250
3/3 [==============================] - 0s 9ms/step - loss: 2.6471 - accuracy: 0.2690
Epoch 10/250
3/3 [==============================] - 0s 8ms/step - loss: 2.6440 - accuracy: 0.2310
Epoch 11/250
3/3 [=====

In [42]:
def sample(probs, temperature=1.0):
    # Cast a float64 por motivos numéricos
    probs = np.asarray(probs).astype('float64')

    # logaritmo de probabilidades y aplicamos reducción
    # por temperatura.
    probs = np.log(probs) / temperature

    # Volvemos a aplicar exponencial y normalizamos de nuevo
    exp_probs = np.exp(probs)
    probs = exp_probs / np.sum(exp_probs)

    # Hacemos el sampling dadas las nuevas probabilidades
    # de salida (ver doc. de np.random.multinomial)
    samples = np.random.multinomial(1, probs, 1)
    return np.argmax(samples)


In [43]:
import random
import sys

TEMPERATURES_TO_TRY = [0.2] #, 0.5, 1.0, 1.2]
GENERATED_TEXT_LENGTH = 300

def generate_text(seed_text, model, length=300, temperature=1, max_length=30):
    """Genera una secuencia de texto a partir de seed_text utilizando model.

    La secuencia tiene longitud length y el sampling se hace con la temperature
    definida.
    """

    # Aquí guardaremos nuestro texto generado, que incluirá el
    # texto origen
    generated = seed_text

    # Utilizar el modelo en un bucle de manera que generemos
    # carácter a carácter. Habrá que construir los valores de
    # X_pred de manera similar a como hemos hecho arriba, salvo que
    # aquí sólo se necesita una oración
    # Nótese que el x que utilicemos tiene que irse actualizando con
    # los caracteres que se van generando. La secuencia de entrada al
    # modelo tiene que ser una secuencia de tamaño SEQ_LENGTH que
    # incluya el último caracter predicho.

    prediction = []

    for i in range(length):
        # Make numpy array to hold seed
        X = np.zeros((1, len(generated), len(chars) ))

        # Set one-hot vectors for seed sequence
        for t, char in enumerate(seed_text):
            X[0, t, char_indices[char]] = 1

        # Generate prediction for next character
        preds = model.predict(X, verbose=0)[0]
        # Choose a character from the prediction probabilities
        next_index = sample(preds,0.2)
        next_char = indice_char[next_index]

        prediction.append(next_char)
        # Add the predicted character to the seed sequence so the next prediction
        # includes this character in it's seed.
        #generated += next_char
        seed_text = seed_text[1:] + next_char

        print(next_char, end= " ")
        # Flush so we can see the prediction as it's generated
        sys.stdout.flush()

    prediction = ''.join(prediction)
    sys.stdout.flush()

    return generated


def on_epoch_end(epoch, logs):
  print("\n\n\n")

  # Primero, seleccionamos una secuencia al azar para empezar a predecir
  # a partir de ella
  start_pos = random.randint(0, len(text) - SEQ_LENGTH - 1)
  seed_text = text[start_pos:start_pos + SEQ_LENGTH]
  for temperature in TEMPERATURES_TO_TRY:
    print("------> Epoch: {} - Generando texto con temperature {}".format(
        epoch + 1, temperature))

    generated_text = generate_text(seed_text, model,
                                   GENERATED_TEXT_LENGTH, temperature)
    print("Seed: {}".format(seed_text))
    print("Texto generado: {}".format(generated_text))


generation_callback = LambdaCallback(on_epoch_end=on_epoch_end)


In [45]:
## TU CÓDIGO AQUÍ

model.fit(X, y, batch_size=128, epochs=5, callbacks=generation_callback)

Epoch 1/5
1/3 [=========>....................] - ETA: 0s - loss: 0.1456 - accuracy: 0.9844



------> Epoch: 1 - Generando texto con temperature 0.2
, ,       s d d     p p s o o o o o r p s a r r v v v e v f a a o d d d t   t g   g o r u o l l l l m l m . e e e v c c c c c c e e c c c c c c e e d d       d d d   n t t a t a r r r r r r           b b b l l l m n m n m l m m           s s i i h h h . . r       e e e h u u o o o o o o l t n t n n n n     a a a a a a a a e e e     e e e u       o o o       s s o s o o o o u u u o i s c c c c c c c c c c e       d c o   d o t o t           n n n n r r t t         a a a a n n n n n n n t         a a a a a a a q q o o o o o o     o o s o o o l l l   u u u u u u u u u u u u l l l l l z n t t n t Seed: idumbre. le encantaban los caballos
Texto generado: idumbre. le encantaban los caballos
3/3 [==============================] - 19s 9s/step - loss: 0.1328 - accuracy: 0.9828
Epoch 2/5
1/3 [=========>....................] - ETA: 0s - loss: 0.1298 

## Conclusiones
Entrenamos un modelo LSTM con un accuracy del 98% aproximadamente, sin embargo, el texto que escogimos es muy corto y aún falta precisión en cada época, la predicción en la época 5 fue "n los caballos, y todas las noches " que tiene sentido gramatical.